# Imports

In [ ]:
import pandas as pd
import numpy as np
import transformers
import torch.nn as nn
from tqdm import tqdm
import torch
from sklearn import model_selection
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from transformers import RobertaConfig

In [ ]:
df=pd.read_csv("../input/commonlitreadabilityprize/train.csv")

# Configuration

In [ ]:
MAX_LEN=256
TEST_BATCH_SIZE=4
TOKENIZER=transformers.RobertaTokenizerFast.from_pretrained("../input/roberta-new")
Roberta_PATH = "../input/roberta-full-training/model.bin"
TRAINING_FILE="../input/commonlitreadabilityprize/train.csv"

# Create the Dataset

In [ ]:
class RobertaDataset:
    def __init__(self,excerpt,target):
        self.excerpt=excerpt
        self.target=target

        
    def __len__(self):
        return len(self.excerpt)
    
    def __getitem__(self,item):
        excerpt=str(self.excerpt[item])
        excerpt=" ".join(excerpt.split())
        
        inputs =TOKENIZER(excerpt,add_special_tokens=True,max_length=MAX_LEN,padding=True,truncation=True)
        
        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]
        
        padding_len=MAX_LEN-len(ids)
        ids=ids+([0]*padding_len)
        mask=mask+([0]*padding_len)
 
        return {"ids": torch.tensor(ids, dtype=torch.long),
            "mask": torch.tensor(mask, dtype=torch.long),
            "targets": torch.tensor(self.target[item], dtype=torch.float)}

# Model

In [ ]:
class RobertaModel(nn.Module):
    
    def __init__(self, conf):
        super(RobertaModel,self).__init__()
        self.roberta=transformers.RobertaModel.from_pretrained(Roberta_PATH,config=conf)
        self.dropout=nn.Dropout(0.3)
        self.linear=nn.Linear(768,1)
        
    def freeze(self):
        for child in self.roberta.children():
            for param in child.parameters():
                param.requires_grad = False

    def unfreeze(self):
        for child in self.roberta.children():
            for param in child.parameters():
                param.requires_grad = True
        
    def forward(self,ids,mask):
        
        output1=self.roberta(ids,attention_mask=mask)
        output1 = output1.hidden_states
        output1 = output1[-1]
        xlnet_output=self.dropout(output1)
        
        out = torch.mean(xlnet_output, 1, False)
        final_output=self.linear(out)
        final_outputs = final_output.squeeze(-1).squeeze(-1)
        
        return final_outputs

# Inference

In [ ]:
def inference(data_loader, model, device):
    model.eval()
    final_loss=0
    i=0
    
    with torch.no_grad():
        for bi, d in tqdm(enumerate(data_loader), total=len(data_loader)):
            ids = d["ids"]
            mask = d["mask"]
            targets = d["targets"]

            ids = ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets = targets.to(device, dtype=torch.float)

            predictions = model(ids=ids, mask=mask)
            predictions = predictions.cpu().detach().numpy()
            
            if i==0:
                preds_test = predictions
            else:
                preds_test = np.concatenate((preds_test,predictions), axis=None)
                
            i+=1

            
    return preds_test

In [ ]:
model_config = RobertaConfig.from_pretrained("../input/roberta-new")

device = torch.device("cuda")
model_config.output_hidden_states = True
model = RobertaModel(model_config)

model.load_state_dict(torch.load(Roberta_PATH))

model.to(device)

test=pd.read_csv("../input/commonlitreadabilityprize/test.csv")
test["target"]=0
test_dataset = RobertaDataset(excerpt=test.excerpt.values, target=test.target.values)

test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=TEST_BATCH_SIZE, num_workers=1)

preds_test=inference(test_data_loader, model, device)

In [ ]:
test.target=preds_test
test=test[["id","target"]]
test.to_csv("submission.csv",index=False)
test.head()